In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\glori\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [3]:
url = 'https://www.zillow.com/va/2_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Virginia%22%2C%22mapBounds%22%3A%7B%22west%22%3A-83.09035759375%2C%22east%22%3A-75.75149040625%2C%22south%22%3A34.320938002477064%2C%22north%22%3A41.53753164462093%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A56%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A7%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%7D'
browser.visit(url)

In [4]:
housePrices = []
houseDetails = []
houseZip = []

for x in range(1, 21):

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    prices = soup.find_all('div', class_='list-card-price')
    details = soup.find_all('ul', class_='list-card-details')
    address = soup.find_all('address', class_="list-card-addr")
    
#     bedroom
#     bathroom
#     sqrft

    for i, price in enumerate(prices):        
        zipcode= address[i].text[len(address[i].text) - 5:]
        
        housePrices.append(price.text)
        houseDetails.append(details[i].text)
        houseZip.append(zipcode)
        
#         print(price.text+'\n', details[i].text+'\n', "zipcode: "+zipcode+'\n')

    browser.find_by_xpath('//*[@id="grid-search-results"]/div[2]/nav/ul/li[10]/a')

In [8]:
house_df = pd.DataFrame(
    {
        'Price': housePrices,
        'Zipcode': houseZip
}


)
house_df

,Price,Zipcode
0,"$240,900",23504
1,"$219,900",23803
2,"$140,000",23456
3,"$274,500",22911
4,"$255,000",23237
...,...,...
175,"$255,000",23237
176,"$310,000",23435
177,"$190,000",23456
178,"$299,900",22821


In [13]:
house_df[house_df.columns[0]] = house_df[house_df.columns[0]].replace('[\$,]', '', regex=True).astype(float)
house_df[house_df.columns[0]] = house_df[house_df.columns[0]].replace('[\+,]', '', regex=True).astype(float)


ValueError: could not convert string to float: '439990+'

In [14]:
house_df.groupby('Zipcode').count()


,Price
Zipcode,
22821,20
22911,20
23237,20
23435,20
23456,40
23504,20
23803,20
house,20


In [15]:
house_df.groupby('Zipcode')['Price'].mean()

DataError: No numeric types to aggregate